# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'11-18-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'11-18-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-19 05:25:23,33.93911,67.709953,43851,1645,35295,6911.0,Afghanistan,112.645437,3.751340
1,NaN,NaN,NaN,Albania,2020-11-19 05:25:23,41.15330,20.168300,29837,646,14216,14975.0,Albania,1036.798944,2.165097
2,NaN,NaN,NaN,Algeria,2020-11-19 05:25:23,28.03390,1.659600,70629,2206,46326,22097.0,Algeria,161.065724,3.123363
3,NaN,NaN,NaN,Andorra,2020-11-19 05:25:23,42.50630,1.521800,6018,76,5055,887.0,Andorra,7788.778878,1.262878
4,NaN,NaN,NaN,Angola,2020-11-19 05:25:23,-11.20270,17.873900,13922,332,6623,6967.0,Angola,42.359540,2.384715


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,42297,42463,42609,42795,42969,43035,43240,43403,43628,43851
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,24731,25294,25801,26211,26701,27233,27830,28432,29126,29837
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,62693,63446,64257,65108,65975,66819,67679,68589,69591,70629


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1574,1577,1581,1591,1595,1605,1617,1626,1638,1645
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,571,579,590,598,605,612,623,631,637,646
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2062,2077,2093,2111,2124,2139,2154,2168,2186,2206


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,34721,34954,34967,35024,35036,35067,35092,35137,35160,35295
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,12203,12353,12493,12574,12667,12767,12889,13453,13804,14216
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,42325,42626,42980,42980,43779,44199,44633,45148,45148,46326


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2328,2328,2351,2385,2417,2435,2456,2481,2506,2529
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,7345,7348,7409,7454,7523,7596,7646,7696,7772,7849
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1107,1107,1112,1113,1117,1123,1128,1130,1134,1137


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,30,30,32,32,36,36,36,36,37,37
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,83,84,84,84,84,84,84,84,84,84
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,9,9,9,9,9,9,9,9,9,10


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
646,1001.0,Autauga,Alabama,US,2020-11-19 05:25:23,32.539527,-86.644082,2529,37,0,2492.0,"Autauga, Alabama, US",4526.660581,1.463029
683,1075.0,Lamar,Alabama,US,2020-11-19 05:25:23,33.779950,-88.096680,596,7,0,589.0,"Lamar, Alabama, US",4317.276349,1.174497
684,1077.0,Lauderdale,Alabama,US,2020-11-19 05:25:23,34.901719,-87.656247,3487,53,0,3434.0,"Lauderdale, Alabama, US",3760.420149,1.519931


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,11527483,250520,4350789
India,8958483,131578,8383602
Brazil,5945849,167455,5383385


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,11527483,250520,4350789,6926174,2020-11-19 05:25:23,37.936303,-91.379001
India,8958483,131578,8383602,443303,2020-11-19 05:25:23,23.088275,81.806127
Brazil,5945849,167455,5383385,395009,2020-11-19 05:25:23,-12.669522,-48.480493
France,2115717,46772,150966,1917979,2020-11-19 05:25:23,8.314863,-21.695174
Russia,1975629,34068,1489893,451668,2020-11-19 05:25:23,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
Texas,1091558,20338,0
California,1064040,18453,0
Florida,905248,17731,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
Texas,1091558,20338,0,1071220,2020-11-19 05:25:23,31.660643,-98.653069
California,1064040,18453,0,1045587,2020-11-19 05:25:23,37.843962,-120.728594
Florida,905248,17731,0,887517,2020-11-19 05:25:23,28.940755,-82.700744
Illinois,606771,11468,0,595303,2020-11-19 05:25:23,39.843618,-89.178330
New York,574072,34187,0,539885,2020-11-19 05:25:23,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,348336,7335,0
Illinois,Cook,262429,6120,0
Florida,Miami-Dade,207221,3731,0
Arizona,Maricopa,180608,3837,0
Texas,Harris,177466,2928,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,348336,7335,0,341001,2020-11-19 05:25:23,34.308284,-118.228241,6037.0
Illinois,Cook,262429,6120,0,256309,2020-11-19 05:25:23,41.841448,-87.816588,17031.0
Florida,Miami-Dade,207221,3731,0,203490,2020-11-19 05:25:23,25.611236,-80.551706,12086.0
Arizona,Maricopa,180608,3837,0,176771,2020-11-19 05:25:23,33.348359,-112.491815,4013.0
Texas,Harris,177466,2928,0,174538,2020-11-19 05:25:23,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-16,43403,28432,68589,5914,13615,134,1318384,117886,27758,208613,...,4104,70381,1,97739,1283,63867,10,2078,17187,8897
2020-11-17,43628,29126,69591,5951,13818,134,1329005,118870,27777,214597,...,4208,70648,1,98050,1288,64935,10,2081,17243,8945
2020-11-18,43851,29837,70629,6018,13922,139,1339337,120459,27785,221688,...,4296,70858,1,98350,1300,66186,10,2083,17280,8981


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-16,1626,631,2168,76,324,4,35727,1788,907,1887,...,67,598,0,855,35,572,1,605,353,257
2020-11-17,1638,637,2186,76,328,4,36106,1811,907,1945,...,68,599,0,858,35,580,1,607,353,260
2020-11-18,1645,646,2206,76,332,4,36347,1839,907,2054,...,68,601,0,860,35,589,1,607,356,261


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-16,35137,13453,45148,4830,6523,127,1140196,75865,25499,129671,...,3284,67655,0,92767,1124,54953,8,1406,16400,8116
2020-11-17,35160,13804,45148,4965,6582,127,1148833,78343,25505,136487,...,3348,67927,0,93084,1124,55684,8,1410,16473,8164
2020-11-18,35295,14216,46326,5055,6623,127,1156474,80069,25507,142738,...,3404,68123,0,93390,1124,56843,8,1417,16495,8184


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2328,2328,2351,2385,2417,2435,2456,2481,2506,2529
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,565,565,570,576,579,581,584,584,588,596
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,3007,3011,3088,3140,3221,3259,3298,3360,3431,3487
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1013,1013,1028,1037,1050,1071,1076,1080,1098,1124
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,6951,6952,6994,7043,7122,7181,7212,7240,7278,7331


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-16,219232,24399,0,276912,134348,1041690,167713,93284,29200,49,...,318888,1066918,155779,3008,1434,204637,131532,34460,334562,23193
2020-11-17,220848,25041,0,279896,135902,1052285,172044,94986,29552,49,...,320729,1078875,158957,3104,1450,206762,134121,35324,342155,24453
2020-11-18,223486,25588,0,283102,137617,1064040,176694,97028,29755,49,...,325201,1091558,162028,3161,1463,208833,135424,36266,350665,25275


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-11-16        2481    7696    1130   993   2574     677   1095    5666   
2020-11-17        2506    7772    1134  1004   2594     678   1099    5702   
2020-11-18        2529    7849    1137  1008   2648     678   1102    5764   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-16         1572      912  ...    1057    260     1321      278   
2020-11-17         1595      919  ...    1092    275     1409      297   
2020-11-18         1620      935  ...    1119    279     1456      311   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-16           1110  1192   760          0      227    338  
2020-11-17           1151  1252   796          0      253    346  
2020-11-18           1224  1271   835          0      263    353  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-16,3249,98,0,6302,2225,18288,2578,4759,736,0,...,3923,20032,723,59,23,3806,2548,585,2764,144
2020-11-17,3301,100,0,6312,2245,18347,2608,4771,739,0,...,3995,20147,732,59,23,3835,2571,598,2867,155
2020-11-18,3347,100,0,6365,2275,18453,2651,4784,742,0,...,4048,20338,740,60,23,3860,2592,612,2925,155


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-16          36      84       9   17     34      19     41      77   
2020-11-17          37      84       9   17     34      19     41      77   
2020-11-18          37      84      10   18     35      19     41      95   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-16           48       15  ...       4      5        8        1   
2020-11-17           48       15  ...       4      5        9        1   
2020-11-18           48       16  ...       4      5        9        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-16              4     2     4          0        7      0  
2020-11-17              4     2     4          0        7      0  
2020-11-18              4     2     4          0        7      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-16,0.003770,0.021631,0.013446,0.007153,0.012192,0.000000,0.006023,0.004679,0.000324,0.022833,...,0.018362,0.001965,0.0,0.003975,0.001561,0.013263,0.0,0.002896,0.003738,0.007702
2020-11-17,0.005184,0.024409,0.014609,0.006256,0.014910,0.000000,0.008056,0.008347,0.000684,0.028685,...,0.025341,0.003794,0.0,0.003182,0.003897,0.016722,0.0,0.001444,0.003258,0.005395
2020-11-18,0.005111,0.024411,0.014916,0.011259,0.007526,0.037313,0.007774,0.013368,0.000288,0.033043,...,0.020913,0.002972,0.0,0.003060,0.009317,0.019265,0.0,0.000961,0.002146,0.004025


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-16,0.005566,0.012841,0.006500,0.0,0.006211,0.0,0.008212,0.014180,0.0,0.031711,...,0.030769,0.000000,NaN,0.004700,0.0,0.012389,0.0,0.000000,0.000000,0.000000
2020-11-17,0.007380,0.009509,0.008303,0.0,0.012346,0.0,0.010608,0.012864,0.0,0.030737,...,0.014925,0.001672,NaN,0.003509,0.0,0.013986,0.0,0.003306,0.000000,0.011673
2020-11-18,0.004274,0.014129,0.009149,0.0,0.012195,0.0,0.006675,0.015461,0.0,0.056041,...,0.000000,0.003339,NaN,0.002331,0.0,0.015517,0.0,0.000000,0.008499,0.003846


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-16,0.001282,0.043758,0.011539,0.017485,0.012259,0.0,0.009826,0.011749,0.001217,0.031805,...,0.015775,0.001910,NaN,0.004265,0.019039,0.011169,0.0,0.008608,0.016298,0.002470
2020-11-17,0.000655,0.026091,0.000000,0.027950,0.009045,0.0,0.007575,0.032663,0.000235,0.052564,...,0.019488,0.004020,NaN,0.003417,0.000000,0.013302,0.0,0.002845,0.004451,0.005914
2020-11-18,0.003840,0.029846,0.026092,0.018127,0.006229,0.0,0.006651,0.022031,0.000078,0.045799,...,0.016726,0.002885,NaN,0.003287,0.000000,0.020814,0.0,0.004965,0.001336,0.002450


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-16,0.006473,0.024565,NaN,0.005359,0.009832,0.009297,0.026289,0.052332,0.013783,0.0,...,0.025571,0.006761,0.012815,0.041191,0.000000,0.013255,0.011473,0.023797,0.014058,0.031075
2020-11-17,0.007371,0.026313,NaN,0.010776,0.011567,0.010171,0.025824,0.018245,0.012055,0.0,...,0.005773,0.011207,0.020401,0.031915,0.011158,0.010384,0.019683,0.025073,0.022695,0.054327
2020-11-18,0.011945,0.021844,NaN,0.011454,0.012619,0.011171,0.027028,0.021498,0.006869,0.0,...,0.013943,0.011756,0.019320,0.018363,0.008966,0.010016,0.009715,0.026667,0.024872,0.033616


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-16      0.010179  0.006539  0.001773  0.007099  0.009808  0.000000   
2020-11-17      0.010077  0.009875  0.003540  0.011078  0.007770  0.001477   
2020-11-18      0.009178  0.009907  0.002646  0.003984  0.020817  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-16      0.007360  0.010342  0.001274  0.004405  ... -0.025806   
2020-11-17      0.003653  0.006354  0.014631  0.007675  ...  0.033113   
2020-11-18      0.002730  0.010873  0.015674  0.017410  ...  0.024725   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-16      0.031746  0.044269  0.037313   0.032558  0.011885  0.042524   
2020-11-17      0.057692  0.066616  0.068345   0.036937  0.050336  0.047368   
2020-11-18      0.014545  0.033357  0.047138   0.063423  0.015176  0.048995   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-16            NaN  0.041284 -0.002950  
2020-11-17            NaN  0.114537  0.023669  
2020-11-18            NaN  0.039526  0.020231  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-16,0.000308,0.000000,NaN,0.000000,0.019240,0.001424,0.012569,0.004644,0.000000,NaN,...,0.007706,0.001850,0.006964,0.000000,0.0,0.001579,0.011513,0.005155,0.004726,0.000000
2020-11-17,0.016005,0.020408,NaN,0.001587,0.008989,0.003226,0.011637,0.002522,0.004076,NaN,...,0.018353,0.005741,0.012448,0.000000,0.0,0.007620,0.009027,0.022222,0.037265,0.076389
2020-11-18,0.013935,0.000000,NaN,0.008397,0.013363,0.005778,0.016488,0.002725,0.004060,NaN,...,0.013267,0.009480,0.010929,0.016949,0.0,0.006519,0.008168,0.023411,0.020230,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga Baldwin   Barbour      Bibb    Blount Bullock Butler   
2020-11-16      0.000000     0.0  0.000000  0.000000  0.000000     0.0    0.0   
2020-11-17      0.027778     0.0  0.000000  0.000000  0.000000     0.0    0.0   
2020-11-18      0.000000     0.0  0.111111  0.058824  0.029412     0.0    0.0   

Province_State                               ... Wyoming                  \
Admin2           Calhoun Chambers  Cherokee  ...    Park Platte Sheridan   
2020-11-16      0.000000      0.0  0.000000  ...     0.0    0.0    0.000   
2020-11-17      0.000000      0.0  0.000000  ...     0.0    0.0    0.125   
2020-11-18      0.233766      0.0  0.066667  ...     0.0    0.0    0.000   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-16          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-11-17          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-11-18          0.0        0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-16,0.003767,0.021109,0.013160,0.011904,0.010364,0.001904,0.006020,0.009174,0.000655,0.029496,...,0.019034,0.002679,0.0,0.004126,0.005426,0.012885,3.113997e-46,0.001482,0.002516,0.006150
2020-11-17,0.004476,0.022759,0.013884,0.009080,0.012637,0.000952,0.007038,0.008761,0.000670,0.029090,...,0.022188,0.003236,0.0,0.003654,0.004662,0.014803,1.556998e-46,0.001463,0.002887,0.005772
2020-11-18,0.004794,0.023585,0.014400,0.010170,0.010082,0.019133,0.007406,0.011064,0.000479,0.031067,...,0.021550,0.003104,0.0,0.003357,0.006989,0.017034,7.784992e-47,0.001212,0.002517,0.004898


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-16,0.005890,0.014038,0.006767,0.003333,0.006033,0.041667,0.006946,0.015933,1.054997e-09,0.038232,...,0.020569,0.002013,NaN,0.004306,1.052711e-24,0.012929,0.0,0.000020,0.002165,0.000561
2020-11-17,0.006635,0.011773,0.007535,0.001667,0.009189,0.020833,0.008777,0.014398,5.274984e-10,0.034484,...,0.017747,0.001843,NaN,0.003907,5.263553e-25,0.013457,0.0,0.001663,0.001083,0.006117
2020-11-18,0.005454,0.012951,0.008342,0.000833,0.010692,0.010417,0.007726,0.014930,2.637492e-10,0.045263,...,0.008874,0.002591,NaN,0.003119,2.631776e-25,0.014487,0.0,0.000832,0.004791,0.004982


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-16,0.001092,0.026267,0.010822,0.015158,0.012322,0.003315,0.009340,0.015580,0.000655,0.041904,...,0.015407,0.002956,NaN,0.004437,0.010439,0.010538,1.824607e-48,0.004358,0.008439,0.001657
2020-11-17,0.000873,0.026179,0.005411,0.021554,0.010683,0.001657,0.008457,0.024122,0.000445,0.047234,...,0.017448,0.003488,NaN,0.003927,0.005220,0.011920,9.123037e-49,0.003602,0.006445,0.003786
2020-11-18,0.002356,0.028013,0.015751,0.019841,0.008456,0.000829,0.007554,0.023077,0.000262,0.046517,...,0.017087,0.003187,NaN,0.003607,0.002610,0.016367,4.561518e-49,0.004283,0.003890,0.003118


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-16,0.008234,0.026778,NaN,0.007607,0.010329,0.009098,0.028676,0.029254,0.013754,6.021063e-75,...,0.021016,0.007586,0.017777,0.033507,0.001343,0.009913,0.014622,0.026044,0.017411,0.031057
2020-11-17,0.007803,0.026545,NaN,0.009191,0.010948,0.009634,0.027250,0.023750,0.012904,3.010532e-75,...,0.013394,0.009397,0.019089,0.032711,0.006250,0.010148,0.017153,0.025558,0.020053,0.042692
2020-11-18,0.009874,0.024195,NaN,0.010323,0.011784,0.010403,0.027139,0.022624,0.009887,1.505266e-75,...,0.013669,0.010576,0.019204,0.025537,0.007608,0.010082,0.013434,0.026113,0.022462,0.038154


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-16      0.009694  0.007096  0.003039  0.007126  0.011645  0.001613   
2020-11-17      0.009885  0.008485  0.003289  0.009102  0.009707  0.001545   
2020-11-18      0.009532  0.009196  0.002967  0.006543  0.015262  0.000772   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-16      0.007700  0.013927  0.005257  0.007808  ... -0.007342   
2020-11-17      0.005677  0.010140  0.009944  0.007742  ...  0.012885   
2020-11-18      0.004203  0.010507  0.012809  0.012576  ...  0.018805   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-16      0.028387  0.036780  0.036158   0.040191  0.020239  0.035802   
2020-11-17      0.043040  0.051698  0.052251   0.038564  0.035287  0.041585   
2020-11-18      0.028792  0.042528  0.049695   0.050994  0.025232  0.045290   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-16           -1.0  0.035980  0.005335  
2020-11-17           -1.0  0.075259  0.014502  
2020-11-18           -1.0  0.057392  0.017367  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-16,0.001871,0.003364,NaN,0.001311,0.013612,0.001525,0.010849,0.002604,0.000945,NaN,...,0.007531,0.003533,0.009908,0.000017,0.000008,0.002013,0.006478,0.009975,0.007723,0.017724
2020-11-17,0.008938,0.011886,NaN,0.001449,0.011300,0.002376,0.011243,0.002563,0.002511,NaN,...,0.012942,0.004637,0.011178,0.000008,0.000004,0.004816,0.007753,0.016099,0.022494,0.047056
2020-11-18,0.011437,0.005943,NaN,0.004923,0.012332,0.004077,0.013865,0.002644,0.003285,NaN,...,0.013104,0.007059,0.011053,0.008479,0.000002,0.005668,0.007960,0.019755,0.021362,0.023528


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-11-16      0.008838  0.000140  2.598556e-13  0.000553  0.004827   
2020-11-17      0.018308  0.000070  1.299278e-13  0.000277  0.002414   
2020-11-18      0.009154  0.000035  5.555556e-02  0.029550  0.015913   

Province_State                                                                \
Admin2           Bullock        Butler   Calhoun      Chambers      Cherokee   
2020-11-16      0.001328  1.192093e-08  0.002196  2.631138e-06  2.724829e-07   
2020-11-17      0.000664  5.960465e-09  0.001098  1.315569e-06  1.362414e-07   
2020-11-18      0.000332  2.980232e-09  0.117432  6.577846e-07  3.333340e-02   

Province_State  ...   Wyoming                                          \
Admin2          ...      Park    Platte  Sheridan Sublette Sweetwater   
2020-11-16      ...  0.041667  0.031321  0.000671      0.0   0.000122   
2020-11-17      ...  0.020833  0.015661  0.062835      0.0   0.000061   
2020-11-18      ...  0.010417  0.007830  0.031418      0.0   0.000031   

Province_State                                                         
Admin2             Teton     Uinta Unassigned      Washakie    Weston  
2020-11-16      0.000061  0.000651  -0.972376  4.850638e-12 -0.533333  
2020-11-17      0.000031  0.000326  -0.972376  2.425319e-12 -0.533333  
2020-11-18      0.000015  0.000163  -0.972376  1.212660e-12 -0.533333  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201118,AK,24419,NaN,861139,NaN,totalTestsViral,885558,147.0,584.0,...,885558,0,22,959c531cf9b1dd6d6e9852ef8674d6e5cc0cde51,0,0,0,0,0,NaN
1,20201118,AL,223486,35863.0,1294045,NaN,totalTestsViral,1481668,1303.0,23088.0,...,1517531,46,372,670bf40671e729deeede1278bfeeb631084588b3,0,0,0,0,0,NaN
2,20201118,AR,137617,15136.0,1415302,NaN,totalTestsViral,1537783,893.0,8139.0,...,1552919,30,89,794c228c7308db198687662757ded2e219d68496,0,0,0,0,0,NaN
3,20201118,AS,0,NaN,1988,NaN,totalTestsViral,1988,NaN,NaN,...,1988,0,0,96485e964f9751aa54a521d8509375693cb3533c,0,0,0,0,0,NaN
4,20201118,AZ,283102,7760.0,1743653,NaN,totalTestsPeopleViral,2018995,1700.0,23473.0,...,2026755,53,230,9e4fa0198456bb08044352368142515709d502ba,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201118,AK,24419.0,NaN,861139.0,NaN,totalTestsViral,885558.0,147.0,584.0,...,885558,0,22,959c531cf9b1dd6d6e9852ef8674d6e5cc0cde51,0,0,0,0,0,NaN
1,20201118,AL,223486.0,35863.0,1294045.0,NaN,totalTestsViral,1481668.0,1303.0,23088.0,...,1517531,46,372,670bf40671e729deeede1278bfeeb631084588b3,0,0,0,0,0,NaN
2,20201118,AR,137617.0,15136.0,1415302.0,NaN,totalTestsViral,1537783.0,893.0,8139.0,...,1552919,30,89,794c228c7308db198687662757ded2e219d68496,0,0,0,0,0,NaN
3,20201118,AS,0.0,NaN,1988.0,NaN,totalTestsViral,1988.0,NaN,NaN,...,1988,0,0,96485e964f9751aa54a521d8509375693cb3533c,0,0,0,0,0,NaN
4,20201118,AZ,283102.0,7760.0,1743653.0,NaN,totalTestsPeopleViral,2018995.0,1700.0,23473.0,...,2026755,53,230,9e4fa0198456bb08044352368142515709d502ba,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-18,AK,24419,NaN,861139,NaN,totalTestsViral,885558,147.0,584.0,NaN,...,885558,0,22,959c531cf9b1dd6d6e9852ef8674d6e5cc0cde51,0,0,0,0,0,NaN
2020-11-18,AL,223486,35863.0,1294045,NaN,totalTestsViral,1481668,1303.0,23088.0,NaN,...,1517531,46,372,670bf40671e729deeede1278bfeeb631084588b3,0,0,0,0,0,NaN
2020-11-18,AR,137617,15136.0,1415302,NaN,totalTestsViral,1537783,893.0,8139.0,339.0,...,1552919,30,89,794c228c7308db198687662757ded2e219d68496,0,0,0,0,0,NaN
2020-11-18,AS,0,NaN,1988,NaN,totalTestsViral,1988,NaN,NaN,NaN,...,1988,0,0,96485e964f9751aa54a521d8509375693cb3533c,0,0,0,0,0,NaN
2020-11-18,AZ,283102,7760.0,1743653,NaN,totalTestsPeopleViral,2018995,1700.0,23473.0,396.0,...,2026755,53,230,9e4fa0198456bb08044352368142515709d502ba,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-18,AK,24419.0,NaN,861139.0,NaN,totalTestsViral,885558.0,147.0,584.0,NaN,...,885558,0,22,959c531cf9b1dd6d6e9852ef8674d6e5cc0cde51,0,0,0,0,0,NaN
2020-11-18,AL,223486.0,35863.0,1294045.0,NaN,totalTestsViral,1481668.0,1303.0,23088.0,NaN,...,1517531,46,372,670bf40671e729deeede1278bfeeb631084588b3,0,0,0,0,0,NaN
2020-11-18,AR,137617.0,15136.0,1415302.0,NaN,totalTestsViral,1537783.0,893.0,8139.0,339.0,...,1552919,30,89,794c228c7308db198687662757ded2e219d68496,0,0,0,0,0,NaN
2020-11-18,AS,0.0,NaN,1988.0,NaN,totalTestsViral,1988.0,NaN,NaN,NaN,...,1988,0,0,96485e964f9751aa54a521d8509375693cb3533c,0,0,0,0,0,NaN
2020-11-18,AZ,283102.0,7760.0,1743653.0,NaN,totalTestsPeopleViral,2018995.0,1700.0,23473.0,396.0,...,2026755,53,230,9e4fa0198456bb08044352368142515709d502ba,0,0,0,0,0,NaN
